In [1]:
import pandas as pd
import numpy as np

In [2]:
BMJ_data_all__b_new = pd.read_csv('../input/BMJ-data-all--b-new.csv', sep='\t')
item_profiles1 = pd.read_csv('../input/item-profiles1.csv', sep=';')
item_profiles2 = pd.read_csv('../input/item-profiles2.csv', sep=';')
item_profiles3 = pd.read_csv('../input/item-profiles3.csv', sep=';')
ratings = pd.read_csv('../input/user-item-rating.csv', sep='\t', names=['user_id','item_id','rating'])

In [3]:
item_profiles = item_profiles1.set_index('Recipe ID').join(item_profiles2.set_index('Recipe ID'), how= 'inner')#.join(item_profiles3.set_index('Recipe ID'), how= 'inner')

In [4]:
def healthiness(itemsDataframe):
    
    fat = ((itemsDataframe['Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    sugar = ((itemsDataframe['Sugar (g)'] * 4) / itemsDataframe['Calories (kCal)']) * 100
    saturatedFat = ((itemsDataframe['Saturated Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    
    itemsDataframe['Unhealtiness'] = (fat / 3) + sugar + saturatedFat
    
    itemsDataframe.loc[fat > 30, 'fatPoints'] = 0
    itemsDataframe.loc[fat >= 40, 'fatPoints'] = -1
    itemsDataframe.loc[fat >= 50, 'fatPoints'] = -2
    itemsDataframe.loc[fat >= 60, 'fatPoints'] = -3
    itemsDataframe.loc[fat >= 70, 'fatPoints'] = -4
    itemsDataframe.loc[fat >= 80, 'fatPoints'] = -5
    itemsDataframe.loc[fat <= 30, 'fatPoints'] = 1 
    itemsDataframe.loc[fat <= 20, 'fatPoints'] = 2
    itemsDataframe.loc[fat <= 10, 'fatPoints'] = 3
    itemsDataframe.loc[fat <= 5, 'fatPoints'] = 4 
    itemsDataframe.loc[fat <= 1, 'fatPoints'] = 5

    itemsDataframe.loc[sugar > 10, 'sugarPoints'] = 0
    itemsDataframe.loc[sugar >= 13, 'sugarPoints'] = -1
    itemsDataframe.loc[sugar >= 16, 'sugarPoints'] = -2
    itemsDataframe.loc[sugar >= 19, 'sugarPoints'] = -3
    itemsDataframe.loc[sugar >= 22, 'sugarPoints'] = -4
    itemsDataframe.loc[sugar >= 25, 'sugarPoints'] = -5
    itemsDataframe.loc[sugar <= 10, 'sugarPoints'] = 1 
    itemsDataframe.loc[sugar <= 7, 'sugarPoints'] = 2
    itemsDataframe.loc[sugar <= 5, 'sugarPoints'] = 3 
    itemsDataframe.loc[sugar <= 3, 'sugarPoints'] = 4
    itemsDataframe.loc[sugar <= 1, 'sugarPoints'] = 5 
        
    itemsDataframe.loc[saturatedFat > 10, 'satFatPoints'] = 0
    itemsDataframe.loc[saturatedFat >= 13, 'satFatPoints'] = -1
    itemsDataframe.loc[saturatedFat >= 16, 'satFatPoints'] = -2
    itemsDataframe.loc[saturatedFat >= 19, 'satFatPoints'] = -3
    itemsDataframe.loc[saturatedFat >= 22, 'satFatPoints'] = -4
    itemsDataframe.loc[saturatedFat >= 25, 'satFatPoints'] = -5
    itemsDataframe.loc[saturatedFat <= 10, 'satFatPoints'] = 1 
    itemsDataframe.loc[saturatedFat <= 7, 'satFatPoints'] = 2
    itemsDataframe.loc[saturatedFat <= 5, 'satFatPoints'] = 3 
    itemsDataframe.loc[saturatedFat <= 3, 'satFatPoints'] = 4
    itemsDataframe.loc[saturatedFat <= 1, 'satFatPoints'] = 5 
        
       
    itemsDataframe['Healthiness'] = itemsDataframe['fatPoints'] + itemsDataframe['satFatPoints'] + itemsDataframe['sugarPoints']

In [5]:
healthiness(item_profiles)

In [6]:
from collections import defaultdict
from sklearn.metrics import ndcg_score

def precision_recall_ndcg_at_k(predictions):
    """Return precision and recall at k metrics for each user"""
    
    k = 10
    threshold = 3.5
    
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:k]
        
    top_k = top_n.copy()
    for uid, user_ratings in top_k.items():
        if (len(top_k[uid]) != 10):
            top_n.pop(uid)
            
            
    healthinessAverage = 0
    count = 0
    for x in top_k.items():
        count =  count + 1
        healthiness = 0
        for y in x[1]:
            healthiness = healthiness + item_profiles.at[y[0], 'Healthiness']
        healthinessAverage = healthinessAverage + (healthiness/10)
    
    healthinessAverage = healthinessAverage/count
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
    predictions_list = [[x[3] for x in predictions]]
    true_scores = [[x[2] for x in predictions]]
    
    
    precision = (sum(prec for prec in precisions.values())) / len(precisions)
    recall = (sum(rec for rec in recalls.values())) / len(recalls)
    ndcg = ndcg_score(true_scores, predictions_list, k)
    
    return precision, recall, ndcg, healthinessAverage
    

In [7]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate


#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

#Create the dataset to be used for building the filter
data = Dataset.load_from_df(ratings, reader)

# We'll use the famous SVD algorithm.
svd = SVD()

kf = KFold(n_splits=5)


for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = svd.test(testset)
    precision, recall, ndcg, healthinessAverage = precision_recall_ndcg_at_k(predictions)
    
    print("Precision:", precision, "Recall:", recall, "NDCG_score:", ndcg, "Healthiness:", healthinessAverage)

C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8694247903563925 Recall: 0.9214364014597275 NDCG_score: 0.9737024221453285 Healthiness: -0.4814465408805025


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8707308461442316 Recall: 0.9250939656859525 NDCG_score: 0.9710526315789473 Healthiness: -0.48913385826771605


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8794350706161713 Recall: 0.9239456987143596 NDCG_score: 0.9776315789473683 Healthiness: -0.41984251968503933


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8762508762705912 Recall: 0.9244563014692782 NDCG_score: 0.9750798722044725 Healthiness: -0.49069400630914867
Precision: 0.881004957187922 Recall: 0.924673707672109 NDCG_score: 0.9780219780219779 Healthiness: -0.4587539432176651


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [11]:
def postfilter(predictions, healthinessFactor):
    predictionsDF = pd.DataFrame.from_records(predictions, columns=['uid', 'iid', 'r_ui', 'est', 'details'])
    ratingsProcessed = predictionsDF.copy()
    ratingsProcessed = ratingsProcessed.join(item_profiles['Healthiness'], how= 'inner', on= 'iid', sort=False)
    ratingsProcessed['est'] = ratingsProcessed['est'] + (ratingsProcessed['Healthiness']*healthinessFactor)
    ratingsProcessed.drop(labels='Healthiness', axis=1, inplace=True)
    ratingsProcessed = list(ratingsProcessed.itertuples(name='Prediction', index=False))
    return ratingsProcessed

In [16]:

#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

#Create the dataset to be used for building the filter
data = Dataset.load_from_df(ratings, reader)

# We'll use the famous SVD algorithm.
svd = SVD()

kf = KFold(n_splits=5)


for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = postfilter(svd.test(testset), 0.0)
    precision, recall, ndcg, healthinessAverage = precision_recall_ndcg_at_k(predictions)
    
    print("Precision:", precision, "Recall:", recall, "NDCG_score:", ndcg, "Healthiness:", healthinessAverage)

C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8765023122109725 Recall: 0.9218842051615227 NDCG_score: 0.9758957654723126 Healthiness: -0.4585039370078749


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.869426634170728 Recall: 0.9243797874909778 NDCG_score: 0.9741741741741738 Healthiness: -0.46204724409448755


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8755400452799978 Recall: 0.9240878071106279 NDCG_score: 0.976190476190476 Healthiness: -0.48195429472025225


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Precision: 0.8737018677081757 Recall: 0.9209768697202522 NDCG_score: 0.9783050847457625 Healthiness: -0.46553627760252325
Precision: 0.8762162196102056 Recall: 0.9243176648941437 NDCG_score: 0.9724409448818896 Healthiness: -0.44912974683544254


C:\tools\miniconda3\envs\recommender\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass k=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
